# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf,  sum,countDistinct, when, col,count
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import  MinMaxScaler, StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import  RandomForestClassifier,LogisticRegression, GBTClassifier
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
get_ipython().run_line_magic('matplotlib', 'inline')

In [3]:
#Spark session
spark_session = (SparkSession.builder
                 .master("local")
                 .appName("Creating Features")
                 .getOrCreate())

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
event_data = spark_session.read.json('mini_sparkify_event_data.json')
event_data.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [5]:
event_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [6]:
event_data = event_data.drop('firstName', 'lastName', 'auth', 'gender', 'artist','song',
                       'method','status', 'registration', 'location','itemInSession')
event_pivot_element = event_data.groupby(["userId"]).pivot("page").count().fillna(0)
event_pivot_element = event_pivot_element.drop( 'Cancel', 'About',  'Submit Registration', 'Login', 'Save Settings','Register')
# filter log to have next song
events_songs = event_data.filter(event_data.page == 'NextSong')
# Total length of songs played
total_song_length = events_songs.groupby(events_songs.userId).agg(sum('length'))
# events_pivot join
event_pivot_element = (event_pivot_element.join(total_song_length, on ='userId', how ='left')
                       .withColumnRenamed("Cancellation Confirmation", "Churn")
                       .withColumnRenamed("sum(length)", "total_length"))
conversion_factor = 24 * 60 * 60 * 1000 # conversion factor to days
# Find min/max time stamp of each user
timestamp_minimum = event_data.select(["userId", "ts"]).groupby("userId").min("ts")
timestamp_maximum = event_data.select(["userId", "ts"]).groupby("userId").max("ts")
# days_active for each user
days_user_active = timestamp_minimum.join(timestamp_maximum, on="userId")
days_user_active = (days_user_active.withColumn("days_active",
                                                (col("max(ts)")-col("min(ts)")) / conversion_factor))
days_user_active = days_user_active.select(["userId", "days_active"])
# events join pivot
event_pivot_element = event_pivot_element.join(days_user_active, on ='userId', how ='left')
# min/max timestamp for each user as paid one
timestamp_min_paid = event_data.filter(event_data.level == 'paid').groupby("userId").min("ts")
timestamp_max_paid = event_data.filter(event_data.level == 'paid').groupby("userId").max("ts")
# Finding days_paid user of each user

days_paid = timestamp_min_paid.join(timestamp_max_paid, on="userId")
days_paid = (days_paid.withColumn("days_paid",
                                  (col("max(ts)")-col("min(ts)")) / conversion_factor))
days_paid = days_paid.select(["userId", "days_paid"])
# joining events pivot
event_pivot_element = event_pivot_element.join(days_paid, on ='userId', how='left')
# min/max time stamp for each user as paid one
timestamp_min_free = event_data.filter(event_data.level == 'free').groupby("userId").min("ts")
timestamp_max_free = event_data.filter(event_data.level == 'free').groupby("userId").max("ts")
# Finding days as paid user of each user
days_free_usage = timestamp_min_free.join(timestamp_max_free, on="userId")
days_free_usage = (days_free_usage.withColumn("days_free",
                                              (col("max(ts)")-col("min(ts)")) / conversion_factor))
days_free_usage = days_free_usage.select(["userId", "days_free"])
# joining events pivot
event_pivot_element = event_pivot_element.join(days_free_usage, on ='userId', how='left')
# finding user_access agents and performing one-hot encoding on the user 
user_access_agents = event_data.select(['userId', 'userAgent']).distinct()
user_access_agents = user_access_agents.fillna('Unknown')
# building string indexer
string_indexing_agent = StringIndexer(inputCol="userAgent", outputCol="userAgentIndex")
generated_model = string_indexing_agent.fit(user_access_agents)
user_access_agents = generated_model.transform(user_access_agents)
# one hot encode userAgent column
one_hot_encoder = OneHotEncoder(inputCol="userAgentIndex", outputCol="userAgentVec")
user_access_agents = one_hot_encoder.transform(user_access_agents).select(['userId', 'userAgentVec'])
# events pivot join
event_pivot_element = event_pivot_element.join(user_access_agents, on ='userId', how ='left')
event_pivot_element = event_pivot_element.fillna(0)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [7]:
# Spliting data into test and train set
event_pivot_element = event_pivot_element.withColumnRenamed('Churn', 'label')
training_split, test_split = event_pivot_element.randomSplit([0.8, 0.2])
# Creating vector from featured data
feature_identifiers = event_pivot_element.drop('label', 'userId').schema.names
assembler_vector = VectorAssembler(inputCols = feature_identifiers, outputCol ="Features")
# Scaling each column
scaling_factor = MinMaxScaler(inputCol="Features", outputCol="ScaledFeatures")
# building classifiers
log_reg = LogisticRegression(featuresCol="ScaledFeatures", labelCol="label",
                             maxIter=10, regParam=0.01)
rf_classifier = RandomForestClassifier(featuresCol="ScaledFeatures", labelCol="label",
                                       numTrees = 50, featureSubsetStrategy='sqrt')
gbt_classifier = GBTClassifier(featuresCol="ScaledFeatures", labelCol="label")
# Consturction of pipelines
pipeline_log_reg = Pipeline(stages=[assembler_vector, scaling_factor, log_reg])
pipeline_rf_classifier = Pipeline(stages=[assembler_vector, scaling_factor, rf_classifier])
pipeline_gbt_classifier = Pipeline(stages=[assembler_vector, scaling_factor, gbt_classifier])

log_reg_model = pipeline_log_reg.fit(training_split)
rf_classifier_model = pipeline_rf_classifier.fit(training_split)
gbt_classifier_model = pipeline_gbt_classifier.fit(training_split)

def evaluate_model(input_model, eval_metric, input_data):
    
    # generating predictions
    evaluator_obj = MulticlassClassificationEvaluator(metricName = eval_metric)
    prediction_op = input_model.transform(input_data)
    
    # calcualting score
    evaluation_output = evaluator_obj.evaluate(prediction_op)
    generated_conf_matrix = (prediction_op.groupby("label")
                                   .pivot("prediction")
                                   .count()
                                   .toPandas())
    return [evaluation_output, generated_conf_matrix]

f1_log_reg, conf_mtx_lr = evaluate_model(log_reg_model, 'f1', test_split)
f1_rf_classifier, conf_mtx_rf = evaluate_model(rf_classifier_model, 'f1', test_split)
f1_gbt_classifier, conf_mtx_gbt = evaluate_model(gbt_classifier_model, 'f1', test_split)
print('The F1 score for the random forest model:', f1_rf_classifier)
conf_mtx_rf

The F1 score for the random forest model: 0.7592592592592593


,label,0.0,1.0
0,0,28,1
1,1,6,1


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.